# Compute the Resource Requirements for Gene Expression in Cell-Free Systems
Fill me in

In [1]:
include("Include.jl");

## Task 1: Build the System Matricies
In this task, we will build the system matrices for the cell-free system. The system matrices are used to describe the interactions between the different components of the system. 

In [2]:
genesequence, proteinsequence = let

    # setup the paths to the gene and protein sequence files
    path_to_gene_file = joinpath(_PATH_TO_DATA, "deGFP.gene")
    path_to_protein_file = joinpath(_PATH_TO_DATA, "deGFP.prot")

    # load the sequences from the files - 
    gene_sequence = load_gene_sequence_from_file(path_to_gene_file); # load the gene sequence
    protein_sequence = load_protein_sequence_from_file(path_to_protein_file); # load the protein sequence

    # return -
    gene_sequence, protein_sequence
end;

Fill me in


In [3]:
reactions = let

    # initialize -
    reactions = Vector{String}();
    
    # TXTL and "hypothetical" exchange reactions -
    transcription_reactions = transcription(genesequence); # transcription reactions
    translation_reactions = translation(proteinsequence); # translation reactions
    txtl_reactions = [transcription_reactions; translation_reactions]; # combine transcription and translation reactions
    
    # build the exchange reactions -
    exchange_reactions = exchangereactions(txtl_reactions); # exchange reactions
    
    # build the list of reactions -
    reactions = [txtl_reactions; exchange_reactions]; # combine all reactions
end;

Fill me in

In [4]:
let

    # initialize -
    path_to_reaction_file = joinpath(_PATH_TO_DATA, "deGFP.reactions"); # path to the reaction file

    # write the reactions to the file -
    open(path_to_reaction_file, "w") do io
        for reaction in reactions
            println(io, reaction); # write the reaction to the file
        end
    end

end

Fill me in.

In [5]:
S, species, reactions, rd, boundsarray = let

    # initialize -
    path_to_reaction_file = joinpath(_PATH_TO_DATA, "deGFP.reactions"); # path to the reaction file
    reactions = read_reaction_file(path_to_reaction_file); # read the reactions from the file
    
    # Compute the stoichiometric matrix -
    (S, species_array, reaction_array, reaction_dictionary) = build_stoichiometric_matrix(reactions); # compute the stoichiometric matrix
   
    # compute the bounds -
    bounds = build_default_bounds_array(reactions); # this is the default bounds array

    # return -
    S, species_array, reactions, reaction_dictionary, bounds
end;

Constants

In [6]:
R = 8.314; # universal gas constant (units: J/(mol*K))
T = 37 + 273.15; # temperature (units: K);
β = 1/(R*T); # inverse temperature (units: 1/(J*mol)) - thermodynamic beta in units of 1/(J*mol)
μ = 0.0; # in this case no growth, we are cell free!
dilution_factor = 30.0; # dilution factor (units: dimensionless)
parameters = generate_parameter_dictionary(joinpath(_PATH_TO_CONFIGURATION, "Parameters.json")); # load the biophysical parameters (approximately true)
INDUCER = 0.1; # initial concentration of the inducer (units: mM)
KIX = 10.0; # Constant for the Hill function in the u-function (units: mM)

To store all the problem data, we create an instance of [the `MyPrimalFluxBalanceAnalysisCalculationModel` type](src/Types.jl). Let's build one of these objects for our problem and store it in the `model::MyPrimalFluxBalanceAnalysisCalculationModel` variable. We also return the `rd::Dict{String, String}` dictionary, which maps the reaction name field (key) to the reaction string (value).
* __Builder (or factory) pattern__: For all custom types that we make, we'll use something like [the builder software pattern](https://en.wikipedia.org/wiki/Builder_pattern) to construct and initialize these objects. The calling syntax will be the same for all types: [a `build(...)` method](src/Factory.jl) will take the kind of thing we want to build in the first argument, and the data needed to make that type as [a `NamedTuple` instance](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) in the second argument.
* __What's the story with the `let` block__? A [let block](https://docs.julialang.org/en/v1/manual/variables-and-scoping/#Let-Blocks) creates a new hard scope and new variable bindings each time they run. Thus, they act like a private scratch space, where data comes in (is captured by the block), but only what we want to be exposed comes out.

In [7]:
model, reactionnamesmap, inversereactionsnamemap, rnamesarray = let

    # build the FBA model -
    model = build(MyPrimalFluxBalanceAnalysisCalculationModel, (
        S = S, # stoichiometric matrix
        fluxbounds = boundsarray, # these are the *default* bounds; we'll need to update with new info if we have it
        species = species, # list of species. The rows of S are in this order
        reactions = reactions, # list of reactions. The cols of S are in this order
        objective = length(reactions) |> R -> zeros(R), # this is empty, we'll need to set this
    ));

    # build a reaction names = reaction index map
    reactionnamesmap = Dict{String,Int64}()
    inversereactionsnamemap = Dict{Int64,String}();
    for i ∈ eachindex(reactions)
        reactionstring = reactions[i]; # get the reaction string
        components = split(reactionstring, ","); # split the reaction string into components around ,
        rname = components[1]; # get the reaction name
        reactionnamesmap[rname] = i;
        inversereactionsnamemap[i] = rname; # add the reaction name to the inverse map
    end

    # build the reaction names array -
    rnamesarray = Vector{String}(undef, length(reactions)); # initialize the reaction names array
    for i ∈ eachindex(reactions)
        reactionstring = reactions[i]; # get the reaction string
        components = split(reactionstring, ","); # split the reaction string into components around ,
        rname = components[1] |> String; # get the reaction name
        rnamesarray[i] = rname; # add the reaction name to the array
    end

    # return -
    model, reactionnamesmap, inversereactionsnamemap, rnamesarray
end;

## Task 2: Update the Bounds Array
Fill me in

There are pathologies when using flux balance analysis to model gene expression, e.g., translation occurs without transcription, and transcription occurs maximally without inducer $I$, etc. How can we fix this? 

__Answer__: there is a trick with the bounds (that incorporates many things we explored in class) that we can use to fix the gene expression problem:
* [Vilkhovoy M, Horvath N, Shih CH, Wayman JA, Calhoun K, Swartz J, Varner JD. Sequence-Specific Modeling of E. coli Cell-Free Protein Synthesis. ACS Synth Biol. 2018 Aug 17;7(8):1844-1857. doi: 10.1021/acssynbio.7b00465. Epub 2018 Jul 16. PMID: 29944340.](https://www.biorxiv.org/content/10.1101/139774v2)

__Fix__: We (equality) bound a transcription rate $\hat{v}_{i}$ as $\hat{r}_{X, i}u_{i} = \hat{v}_{i} = \hat{r}_{X, i}u_{i}$, while a translation rate is bounded from above by the modified kinetic limit: $0\leq\hat{v}_{j}\leq\hat{r}_{L,j}w_{j}$. This is interesting because the kinetic limits are (semi)mechanistic descriptions of the transcription and translation rate. At the same time, the control variables contain continuous Boltzmann-type descriptions of the logical controlling of these processes.

Let's implement this for our gene expression model. First, let's compute the kinetic limit of transcription for your gene of interest:

In [8]:
rX = compute_transcription_rate(parameters); # transcription kinetic limit

In [9]:
println("Maximim transcription rate (mu/mol-hr): ", rX); # print the transcription rate

Maximim transcription rate (mu/mol-hr): 0.2088758116178622


Next, we'll compute the transcriptional and translational control terms that govern inducer $I$ driven gene expression. For simplicity, let's assume the translation control term is unity $w = 1$ and focus on a Boltzmann-type description of the transcriptional control term $u$. 

For the PhoA promoter, let's assume three states:
* __State 0: Bare gene__: This state is just the gene without anything bound to it. This state will __not__ lead to transcription. This will be our ground state. The pseudo energy for this state $\epsilon_{0} \equiv 0$ J/mol.
* __State 1: RNAP only__: Only RNAP is bound to the promoter without inducer bound. This state __will__ lead to transcription at a low level. The pseudo energy for this state $\epsilon_{1} \approx 3474$ J/mol.
* __State 2: RNAP + I__: In this state both RNAP and inducer $I$ are bound to the promoter. This state __will__ lead to transcription. The pseudo energy for this state $\epsilon_{2} \approx -14,707$ J/mol.

Update the code block below to compute $u_{1}$ (the control variable for state 1), $u_{2}$ (the control variable for state 2), and translation control parameter $w$. 
* _What the what, confused?_ For a reference on how to do this (and what we are talking about), see the Supplemental materials of [Moon et al.](https://pubmed.ncbi.nlm.nih.gov/23041931/) and/or the lecture `L6c` materials.

Update the code block below at the `TODO` statements.

In [10]:
u₁, u₂, w = let 
    
    # initialize -
    w = 1.0;
    u₁ = 0.0; # this is udagger (background)
    u₂ = 0.0; # this is u (induced)
    ϵₒ = 0.0; # pseudo-energy for the background (gene only) state 0
    ϵ₁ = 3474.0; # pseudo-energy for state 1 (gene + RNAP) - units: J/mol
    ϵ₂ = -14707.0; # pseudo-energy for state 2 (gene + RNAP + P_PhoB) - units: J/mol

    # we'll model the f₂ function as the fraction of PhoB binding. Others are at defaults
    fₒ = 1.0; # default value
    f₁ = 1.0; # default value
    f₂ = INDUCER/(INDUCER + KIX); # hmmm. This is sort of interesting ...

    # compute the weights of each state
    Wₒ = fₒ*exp(-β*ϵₒ); # TODO: Update the weight term for state 0
    W₁ = f₁*exp(-β*ϵ₁); # TODO: Update the weight term for state 1
    W₂ = f₂*exp(-β*ϵ₂); # TODO: Update the weight term for state 2
    Z = Wₒ + W₁ + W₂; # partition function

    # Boltzmann promoter logic here -
    u₁ = W₁/Z; # TODO: Update u₁ (this is u dagger in the notes)
    u₂ = W₂/Z; # TODO: Update u₂ (this is u)
    
    # return -
    u₁,u₂,w
end;

In [11]:
println("u₁ (background expression factor): $(u₁) and u₂ (induced expression) $(u₂)"); # print the u₁ and u₂ value

u₁ (background expression factor): 0.06146297872016685 and u₂ (induced expression) 0.7020993915068997


Finally, let's compute the kinetic limit of translation. This one is tricky because it requires an estimate of the _concentration_ of the mRNA for gene of interest.

See lecture `L5b` for a description of the kinetic limit of translation expression (or the reference we gave above). Let's approximate the mRNA level by the steady-state level (written for transcript $j$):
$$
\begin{align*}
 m^{\star}_{j} & = \frac{r_{X,j}u_{j}\left(\dots\right) + \lambda_{j}}{\theta_{m,j}+\mu}\quad\text{for }j=1,2,\dots,N
\end{align*}
$$
where $\lambda_{j} \equiv r_{X,j}u^{\dagger}_{j}$. Update the code block below at the `TODO` statements.

In [12]:
rL, m̄ = let

    # get constants from parameters -
    θ = parameters[:kdX]; # first order degradation constant mRNA (units: 1/hr)
    KL = parameters[:KL]; # saturation constant translation (units: μmol/gDW-hr)
    VMAXL = parameters[:VL]; # VMAX translation (correct PhoA length) (units: μmol/gDW-hr)
    τ = parameters[:L_tau_factor]; # relative time constant translation (units: dimensionless)
    
    # compute -
    m = (rX*u₁ + rX*u₂)/(θ + μ); # TODO: approx mRNA level with steady-state
    rL = VMAXL*(m/(τ*KL+(1+τ)*m)); # compute the kinetic limit of translation

    # return -
    rL, m
end;

In [13]:
println("Maximim translation rate (mu/mol-hr): ", rL); # print the translation rate

Maximim translation rate (mu/mol-hr): 0.18702150780759302


### Update the bounds
Now that we have an estimate of the transcription $\hat{r}_{X}u$ and the translation $\hat{r}_{L}w$ rates, we can update the flux balance analysis problem bounds. Let's play around with these bounds to see what happens.

* There is nothing for you to do on this block, but if you are interested, please unhide the block and take a look. 

In [14]:
fluxbounds = let

    # make a copy of the default flux bounds -
    flux_bounds = copy(model.fluxbounds);

    # what is the the bound on transcription?
    XB = rX*(u₁ + u₂); # this is the lower bound on transcription

    # update the bounds for gene exprssion -
    flux_bounds[reactionnamesmap["transcription_test"], 1] = XB; # transcrption lower bound state 1
    flux_bounds[reactionnamesmap["transcription_test"], 2] = XB; # transcrption upper bound state 1

    flux_bounds[reactionnamesmap["translation_initiation_test"], 1] = rL*w; # translation lower bound
    flux_bounds[reactionnamesmap["translation_initiation_test"], 2] = rL*w; # translation upper bound
    flux_bounds[reactionnamesmap["translation_test"], 1] = 0.0; # translation lower bound
    flux_bounds[reactionnamesmap["translation_test"], 2] = rL*w; # translation upper bound

    # update bounds on tRNA - no exchange of charged tRNA
    tRNA_exchange_reactions = findall(x-> (contains(x, "exchange") && contains(x,"tRNA") && contains(x, "_M_")), rnamesarray); # find the index of the reaction
    for i ∈ eachindex(tRNA_exchange_reactions)
        flux_bounds[tRNA_exchange_reactions[i], 1] = 0.0; # no charged tRNA from box
        flux_bounds[tRNA_exchange_reactions[i], 2] = 0.0; # no charged tRNA into box
    end

    # bound mRNA degradation -
    θ = parameters[:kdX]; # first order degradation constant mRNA (units: 1/hr)
    flux_bounds[reactionnamesmap["mRNA_degradation_test"], 1] = 0.0 # lower bound on mRNA degradation
    flux_bounds[reactionnamesmap["mRNA_degradation_test"], 2] = θ*m̄; # upper bound on mRNA degradation

    # no mRNA exchange -
    flux_bounds[reactionnamesmap["exchange_mRNA_test"], 1] = 0.0; # lower bound on mRNA exchange
    flux_bounds[reactionnamesmap["exchange_mRNA_test"], 2] = 0.0; # upper bound on mRNA exchange

    # return new bounds
    flux_bounds;
end;

In [15]:
model.fluxbounds = fluxbounds; # update the flux bounds in the model

## Task 3: Estimate the Protein Production Rate
In this task, we will estimate the protein production rate in a cell-free system as a function of inducer concentration. Let's begin by updating the objective function.

In [16]:
objective_coefficients = let

    # initialize -
    nreactions = length(model.reactions); # how many reactions do we have?
    objective_coefficients = zeros(nreactions); # initialize the objective coefficients

    # which reactions do we want to maximize?
    reactions_to_maximize = ["translation_test"]; # TODO: Add reactions that we want to max
    for i ∈ eachindex(reactions_to_maximize)
        reaction = reactions_to_maximize[i];
        j = reactionnamesmap[reaction];
        objective_coefficients[j] = 1;
    end

    # return -
    objective_coefficients;
end;

In [17]:
model.objective = objective_coefficients; # update the objectivec coefficients

### Compute the optimal flux distribution 
Next, let's compute the optimal metabolic distribution $\left\{\hat{v}_{i} \mid i = 1,2,\dots,\mathcal{R}\right\}$ by solving the [linear programming problem](). We solve the optimization problem by passing the `model::MyPrimalFluxBalanceAnalysisCalculationModel` to [the `solve(...)` method](src/Compute.jl). This method returns a `solution::Dict{String, Any}` dictionary, which holds information about the solution.
* __Why the [try-catch environment](https://docs.julialang.org/en/v1/base/base/#try)__? The [solve(...) method](src/Compute.jl) has an [@assert statement](https://docs.julialang.org/en/v1/base/base/#Base.@assert) to check if the calculation has converged. Thus, the solve method can [throw](https://docs.julialang.org/en/v1/base/base/#Core.throw) an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) if the optimization problem fails to converge. To gracefully handle this case, we use a [try-catch construct](https://docs.julialang.org/en/v1/base/base/#try). See the [is_solved_and_feasible method from the JuMP package](https://jump.dev/JuMP.jl/stable/api/JuMP/#JuMP.is_solved_and_feasible) for more information.

In [18]:
solution = let
    
    solution = nothing; # initialize nothing for the solution
    try
        solution = solve(model); # call the solve method with our problem model -
    catch error
        println("error: $(error)"); # Oooooops! Looks like we have a *major malfunction*, problem didn't solve
    end

    # return solution
    solution
end;

### Flux table

If the optimization problem converged, we should have the optimal flux values throughout the system. Let's use [the `pretty_tables(...)` method exported by the `PrettyTables.jl` package](https://github.com/ronisbr/PrettyTables.jl) to display the estimated optimal metabolic fluxes. `Unhide` the code block below to see how we constructed the flux table.

In [21]:
let

    # setup -
    S = model.S;
    flux_bounds_array = model.fluxbounds;
    number_of_reactions = size(S,2); # columns
    flux = solution["argmax"];

	# nonzero fluxes -
	nonzero_fluxes = findall(v-> abs(v) ≥ 1e-6, flux); # find the non-zero fluxes
	number_of_nonzero_fluxes = length(nonzero_fluxes); # how many non-zero fluxes do we have?
    
    # populate the state table -
	flux_table = Array{Any,2}(undef,number_of_nonzero_fluxes,5)
	for reaction_index ∈ eachindex(nonzero_fluxes)
		i = nonzero_fluxes[reaction_index]; # get the index of the reaction
		flux_table[reaction_index,1] = inversereactionsnamemap[i]
		flux_table[reaction_index,2] = flux[i]
		flux_table[reaction_index,3] = flux_bounds_array[i,1]
		flux_table[reaction_index,4] = flux_bounds_array[i,2]
        flux_table[reaction_index,5] = inversereactionsnamemap[i] |> key-> rd[key]
	end

    # header row -
	flux_table_header_row = (["Reaction","v̂ᵢ", "v̂ᵢ LB", "v̂ᵢ UB", "Reaction"],["","μmol/L-time", "μmol/L-time", "μmmol/L-time", "N/A"]);
		
	# write the table -
	pretty_table(flux_table; header=flux_table_header_row, tf=tf_simple, alignment = :l)
end

=============================== ============= ============= ============== =====================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================
  Reaction                       v̂ᵢ            v̂ᵢ LB         v̂ᵢ UB          Reaction                                                                                                                                                                                                                                                                                                                                                

## Discussion

## Tests
`Unhide` the code block below (if you are curious) about how we implemented the tests and what we are testing. In these tests, we check values in your notebook and give feedback on which items are correct, missing etc.

In [ ]:
let
    @testset verbose = true "CHEME 5450 practicum test suite" begin
        
        @testset "Setup" begin
            @test isnothing(model) == false
            @test isnothing(rd) == false
            @test isnothing(reactionnamesmap) == false
        end

        @testset "Problem, Bounds and Optmization" begin
            @test rX != 0.0; # kinetic limit of transcription should be non-zero
            @test rL != 0.0; # kinetic limit of translation should be non-zero
            @test w == 1.0; # default value 
            @test u₁ != 0.0; # background state should be non-zero
            @test u₂ != 0.0; # induced state should be non-zero            
            @test isempty(solution) == false # solution should not be empty
        end
        
        @testset "Discussion questions" begin
            @test did_I_answer_DQ1 == true
            @test did_I_answer_DQ2 == true
            @test did_I_answer_DQ3 == true
        end
    end
end;